# This notebook provides interactive widgets to play with values for the SKA1 SDP

## First, do the necessary set-up

In [1]:
'''
The following commands import the python modules, methods and code fragments that are required to run this notebook. 
Please refer to the following python files for implementation details (omitted here for readability)
* env_setup.py -- code for setting up the python environment
* parameter_definitions.py -- contains definitions of variables, primary telescope parameters  
* formulae.py -- contains formulae for defiving secondary telescope-specific parameters from input parameters
* implementation.py -- contains methods for performing computations (i.e. crunching the numbers)
'''

from IPython.html.widgets import interact, interactive, fixed
from IPython.html.widgets.interaction import interact_manual
from IPython.html import widgets
from IPython.display import clear_output, display, HTML

from env_setup import *
from parameter_definitions import *
from formulae import *
from implementation import *

def show_table(header, titles, values, units):
    s = '<h3>%s:</h3><table>\n' % header
    assert len(titles) == len(values)
    assert len(titles) == len(units)
    for i in range(len(titles)):
        s += '<tr><td>{0}</td><td><font color="blue">{1}</font> {2}</td></tr>\n'.format(titles[i], values[i], units[i])
    s += '</table>'
    display(HTML(s))
    
band_lookup = {'SKA1-Low':'Low', 'SKA1-Mid (Band 1)':'Mid1', 'SKA1-Survey (Band 1)':'Sur1'}
telescopes_pretty_print =('SKA1-Low', 'SKA1-Mid (Band 1)', 'SKA1-Survey (Band 1)')
modes_pretty_print = ('Continuum', 'Spectral', 'SlowTrans')

mode_lookup = {}
for key in modes_pretty_print:
    mode_lookup[key] = key

def compute_flops(max_baseline, Nfacet, Tsnap, Telescope, Mode):    
    band = band_lookup[Telescope]
    titles = ('Max Baseline', 'Telescope', 'Band', 'Mode', 'Tsnap', 'Nfacet')

    values = (max_baseline, Telescope, band, Mode, Tsnap, Nfacet)
    units = ('km', '', '', '', 'sec', '')
    show_table('Arguments', titles, values, units)
    
    telescope_parameters = calc_tel_params(band=band, mode=mode_lookup[Mode])
    max_allowed_baseline = telescope_parameters.baseline_bins[-1] / u.km
    
    if max_baseline <= max_allowed_baseline:
        telescope_parameters.Bmax = max_baseline * u.km

        '''
        (Tsnap_opt, Nfacet_opt) = find_optimal_Tsnap_Nfacet(telescope_parameters, verbose=True)
        '''
        expression = telescope_parameters.Rflop
        expression_subst = expression.subs({telescope_parameters.Tsnap : Tsnap, telescope_parameters.Nfacet : Nfacet})
        result_titles = ('Total Compute Requirement',)
        result_units = ('PetaFLOPS',)
        result = evaluate_binned_expression(expression_subst, telescope_parameters)
        result_values = ('%g' % result,)
        
        show_table('Results', result_titles, result_values, result_units)
    else :
        msg = 'ERROR: max_baseline exceeds the maximum allowed baseline of %g km for this telescope.' % max_allowed_baseline
        s = '<font color="red"><b>{0}</b>.<br>Adjust to recompute.</font>'.format(msg)
        display(HTML(s))
    

IPython console for SymPy 0.7.6 (Python 2.7.7-32-bit) (ground types: python)


:0: FutureWarning: IPython widgets are experimental and may change in the future.


## Now, we can interactively play with parameters

### The first option is automatic updating of results as the sliders are moved. This may be sluggish

In [2]:
interact(compute_flops, max_baseline=(10,200), Nfacet=(1,10,1), Tsnap=(1.2,1800), Telescope=telescopes_pretty_print, 
         Mode=modes_pretty_print);

Max Baseline,39 km
Telescope,SKA1-Mid (Band 1)
Band,Mid1
Mode,Continuum
Tsnap,900.6 sec
Nfacet,5


Total Compute Requirement,2.06905 PetaFLOPS


In [3]:
### The second option is manual triggering of recompute events (recommended)

In [4]:
interact_manual(compute_flops, max_baseline=(10,200), Nfacet=(1,128,1), Tsnap=(1.2,1800), Telescope=telescopes_pretty_print, 
         Mode=modes_pretty_print);

Max Baseline,76 km
Telescope,SKA1-Low
Band,Low
Mode,Continuum
Tsnap,900.6 sec
Nfacet,64


Total Compute Requirement,352.201 PetaFLOPS


# Old code (to be deleted):

In [19]:
telescope_parameters = calc_tel_params(band='Low', mode='Continuum')
#telescope_parameters.Bmax = max_baseline * u.km

(Tsnap_opt, Nfacet_opt) = find_optimal_Tsnap_Nfacet(telescope_parameters, verbose=True)
expression = telescope_parameters.Rflop
expression_subst = expression.subs({telescope_parameters.Tsnap : Tsnap_opt, telescope_parameters.Nfacet : Nfacet_opt})
result = evaluate_binned_expression(expression_subst, telescope_parameters)


Evaluating Nfacets = 1
Tsnap has been optimized as : 36.819960, yielding a minimum value of 0.000000 Peta-units
Evaluating Nfacets = 2
Tsnap has been optimized as : 61.758547, yielding a minimum value of 0.000000 Peta-units
Evaluating Nfacets = 3
Tsnap has been optimized as : 86.900112, yielding a minimum value of 0.000000 Peta-units
Evaluating Nfacets = 4
Tsnap has been optimized as : 109.984848, yielding a minimum value of 0.000000 Peta-units

Expression increasing with number of facets; aborting exploration of Nfacets > 4

0.000000 PetaFLOPS was the lowest FLOP value, found for (Nfacet, Tsnap) = (3, 86.90)


In [5]:
import time
t0 = time.time()
band = 'Low'
mode = 'Spectral'
telescope_parameters = calc_tel_params(band=band, mode=mode)
(Tsnap_opt, Nfacet_opt) = find_optimal_Tsnap_Nfacet(telescope_parameters, verbose=True)
print (Tsnap_opt, Nfacet_opt)
#(Tsnap_opt, Nfacet_opt) = (87.16, 3)
t1 = time.time()
expression = telescope_parameters.Rflop
expression_subst = expression.subs({telescope_parameters.Tsnap : Tsnap_opt, telescope_parameters.Nfacet : Nfacet_opt})
result = evaluate_binned_expression(expression_subst, telescope_parameters)
print 'Result of expression = %f Peta' % (result)
t2 = time.time()
print 'Optimization took %f seconds' % (t1-t0)
print 'Evaluation took %f seconds' % (t2-t1)
print 'Total time = %f seconds' % (t2-t0)

Evaluating Nfacets = 1
Tsnap has been optimized as : 117.742556, yielding a minimum value of 0.000000 Peta-units
Evaluating Nfacets = 2
Tsnap has been optimized as : 164.551134, yielding a minimum value of 0.000000 Peta-units

Expression increasing with number of facets; aborting exploration of Nfacets > 2

0.000000 PetaFLOPS was the lowest FLOP value, found for (Nfacet, Tsnap) = (1, 117.74)
(117.742556400779, 1)
Result of expression = 34.255293 Peta
Optimization took 11.059000 seconds
Evaluation took 0.441000 seconds
Total time = 11.500000 seconds
